In [1]:
import pandas as pd
import os
import datetime
import numpy as np
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
samplerows=None
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/JT_sales_quarter_to_date_20191001'

In [2]:
current_quarter_beginning=datetime.date(2019,8,4)

weekly_daily_df=list(recursive_file_gen("/home/jian/BigLots/"))
weekly_daily_df=[x for x in weekly_daily_df if ("2018" not in x) & ("2017" not in x) & ("2016" not in x) & ("hist" not in x.lower())]
weekly_daily_df=[x for x in weekly_daily_df if (".txt" in x) & ("aily" in x)]

weekly_daily_df=pd.DataFrame({"file_path":weekly_daily_df})
weekly_daily_df['week_end_dt']=weekly_daily_df['file_path'].apply(lambda x: datetime.datetime.strptime(x.split("/MediaStorm_")[1][:10],"%Y-%m-%d").date())
weekly_daily_df=weekly_daily_df[weekly_daily_df['week_end_dt']>current_quarter_beginning]
weekly_daily_df.shape

(8, 2)

In [3]:
current_quarter_file_list=weekly_daily_df['file_path'].tolist()
current_quarter_sales=pd.DataFrame()
for file in current_quarter_file_list:
    week_end_dt=file.split("/MediaStorm_")[1][:10]
    df=pd.read_table(file,sep="|",dtype=str,nrows=samplerows,
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_rewards=df[~pd.isnull(df['customer_id_hashed'])]
    df_nonrewards=df[pd.isnull(df['customer_id_hashed'])]
    
    # Rewards:
    df_sales=df_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"rewards_trans"})
    
    rewards_shoppers=df_rewards.groupby(['location_id','transaction_dt'])['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"shopper_by_store_day"})
    
    df_rewards=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt'],how="outer")
    df_rewards=pd.merge(df_rewards,rewards_shoppers,on=['location_id','transaction_dt'],how="outer")
    
    # Non_Rewards:
    df_sales=df_nonrewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"nonrewards_sales"})
    df_trans=df_nonrewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_trans=df_trans.groupby(['location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"nonrewards_trans"})
    df_nonrewards=pd.merge(df_sales,df_trans,on=['location_id','transaction_dt'],how="outer")
    
    df_week=pd.merge(df_rewards,df_nonrewards,on=['location_id','transaction_dt'],how="outer").fillna(0)
    df_week['week_end_dt']=week_end_dt

    current_quarter_sales=current_quarter_sales.append(df_week)
    print(datetime.datetime.now())

2019-10-02 16:50:52.781200
2019-10-02 16:55:18.935554
2019-10-02 16:58:37.752407
2019-10-02 17:01:51.094254
2019-10-02 17:05:55.204563
2019-10-02 17:08:47.813919
2019-10-02 17:12:12.500470
2019-10-02 17:15:32.644334


In [18]:
current_quarter_sales['Online/Instore']=np.where(current_quarter_sales['location_id']=="6990","Online","Instore")
current_quarter_sales['location_id']=current_quarter_sales['location_id'].astype(int)
current_quarter_sales=current_quarter_sales.sort_values(['location_id','transaction_dt','Online/Instore'])

current_quarter_sales_week=current_quarter_sales.groupby("week_end_dt")['rewards_sales','rewards_trans','nonrewards_sales','nonrewards_trans'].sum().reset_index()
current_quarter_sales_week_type=current_quarter_sales.groupby(["week_end_dt",'Online/Instore'])['rewards_sales','rewards_trans','nonrewards_sales','nonrewards_trans'].sum().reset_index()



In [20]:
writer=pd.ExcelWriter("./BL_weekly_tracker_overall_Rewards_NonRewards_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
current_quarter_sales_week.to_excel(writer,"sales_by_week",index=False)
current_quarter_sales_week_type.to_excel(writer,"sales_by_week_by_type",index=False)
current_quarter_sales.to_excel(writer,"sales_by_day",index=False)
writer.save()